# SEC EDGAR

### Imports and Constants

In [1]:
import asyncio
import concurrent.futures
import json
import os
import random
import time
import zipfile
from datetime import datetime, timedelta
from typing import Dict, List, Set, Tuple, Union

import aiohttp
import numpy as np
import orjson
import pandas as pd
import requests
import smart_open
from bs4 import BeautifulSoup
from dateutil import relativedelta
from smart_open import open
from tqdm import tqdm


In [2]:
heading = {"User-Agent": "locke@gatech.edu"}

### Create ticker to CIK mapping & get SP500 tickers

In [3]:
url = "https://www.sec.gov/files/company_tickers.json"
r = requests.get(url, headers=heading)
ticker_cik = pd.DataFrame(r.json()).T
ticker_cik.set_index("ticker", inplace=True)

In [4]:
def get_cik(ticker: str) -> str:
    return str(ticker_cik.loc[ticker, "cik_str"]).zfill(10)

In [5]:
def get_tickers_and_sectors() -> List[Tuple[str, str]]:
    """Get a list of tickers and their sectors for all stocks in the S&P 500.

    Returns:
        List[Tuple[str, str]]: List of tuples containing ticker and sector.
    """
    url = "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    source = BeautifulSoup(response.text, "lxml")
    table = source.find("table", {"class": "wikitable sortable"})
    tickers_and_sectors = []
    for row in table.findAll("tr")[1:]:
        ticker = row.findAll("td")[0].text.replace("\n", "")
        sector = row.findAll("td")[3].text.replace("\n", "")
        tickers_and_sectors.append((ticker, sector))
    return tickers_and_sectors

### Download and process bulk data from SEC

In [6]:
DATA_DIR = "data"
RAW_DATA_DIR = os.path.join(DATA_DIR, "raw")
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed")

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RAW_DATA_DIR, exist_ok=True)
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

In [7]:
def download_data():
    """
    Download all company facts from SEC.
    """
    # if data/companyfacts.zip does not exist, download it
    if not os.path.exists(os.path.join(RAW_DATA_DIR, "companyfacts.zip")):
        print("Downloading companyfacts.zip...")
        url = "https://www.sec.gov/Archives/edgar/daily-index/xbrl/companyfacts.zip"
        r = requests.get(url, headers=heading)
        with open("data/raw/companyfacts.zip", "wb") as f:
            f.write(r.content)
    # if JSON files do not exist, unzip them
    if not os.path.exists(os.path.join(RAW_DATA_DIR, "CIK000032019.json")):
        print("Unzipping companyfacts.zip...")
        with zipfile.ZipFile(os.path.join(RAW_DATA_DIR, "companyfacts.zip"), "r") as zip_ref:
            zip_ref.extractall("data/raw")

In [8]:
def process_item(item: Dict) -> Dict:
    """
    Process a single item in the SEC company facts JSON file.

    Args:
        item (Dict): The item to process, e.g. "dei:EntityCommonStockSharesOutstanding".

    Returns:
        Dict: The processed item.
    """

    processed_item = {}
    processed_item["label"] = item["label"]
    processed_item["description"] = item["description"]

    for unit in item["units"]:

        # store data by timeframe (i.e. FY2022, Q32019) and filing date
        # for each category, seperate annual data from quarterly data
        processed_item[unit] = {}
        processed_item[unit]["by_timeframe"] = {"annual": {}, "quarterly": {}}
        processed_item[unit]["by_filing_date"] = {"annual": {}, "quarterly": {}}
        for entry in item["units"][unit]:
            fy = entry["fy"]
            fp = entry["fp"]
            filing_date = entry["filed"]
            if entry["fp"] == "FY":
                processed_item[unit]["by_timeframe"]["annual"][fy] = entry
                processed_item[unit]["by_filing_date"]["annual"][filing_date] = entry
            else:
                if fy not in processed_item[unit]["by_timeframe"]["quarterly"]:
                    processed_item[unit]["by_timeframe"]["quarterly"][fy] = {}
                processed_item[unit]["by_timeframe"]["quarterly"][fy][fp] = entry
                processed_item[unit]["by_filing_date"]["quarterly"][filing_date] = entry

    return processed_item

In [9]:
def process_json(ticker: str) -> None:
    """
    Process the SEC company facts JSON file for a given ticker, creating a new JSON file.

    Args:
        ticker (str): The ticker symbol for the company to process.
    """

    # find SEC data for the company
    cik = get_cik(ticker)
    try:
        with open(os.path.join(RAW_DATA_DIR, f"CIK{cik}.json")) as f:
            sec_data = orjson.loads(f.read())
    except FileNotFoundError:
        raise FileNotFoundError(f"CIK{cik}.json not found in raw data directory. Use download_data().")

    # process the data
    data = {"ticker": ticker, "cik": sec_data["cik"], "entityName": sec_data["entityName"]}
    for item in sec_data["facts"]["dei"]:
        data[item] = process_item(sec_data["facts"]["dei"][item])
    for item in sec_data["facts"]["us-gaap"]:
        data[item] = process_item(sec_data["facts"]["us-gaap"][item])

    # write the data to a JSON file
    with open(os.path.join(PROCESSED_DATA_DIR, f"{ticker}.json"), "w") as f:
        f.write(json.dumps(data, indent=4))

Only run the below cells if you want to download and process the most recent data from the SEC.

In [10]:
# get up to date data from SEC
download_data()

Unzipping companyfacts.zip...


In [ ]:
# process data for stocks in S&P 500
tickers_and_sectors = get_tickers_and_sectors()
for ticker, sector in tqdm(tickers_and_sectors):
    try:
        process_json(ticker)
    except Exception as e:
        print(f"Could not process data for {ticker}. Error: {e}")

### Generic methods to retrieve metrics from processed JSON data

In [11]:
def get_metric_by_timeframe(ticker: str, metric: str, units: str, year: str, quarter: str = None) -> float:
    """
    Get a metric for a company by timeframe.

    Args:
        ticker (str): The ticker symbol for the company.
        metric (str): The metric to get.
        year (str): The year to get the metric for.
        quarter (str, optional): The quarter to get the metric for. Defaults to None, which gets the annual metric.

    Returns:
        float: The metric value.
    """

    with open(os.path.join(PROCESSED_DATA_DIR, f"{ticker}.json")) as f:
        data = orjson.loads(f.read())

    try:
        if quarter is None:
            return data[metric][units]["by_timeframe"]["annual"][year]["val"]
        else:
            return data[metric][units]["by_timeframe"]["quarterly"][year][quarter]["val"]
    except KeyError:
        if quarter is None:
            raise KeyError(f"Metric {metric} not found for {ticker} in {year}.")
        else:
            raise KeyError(f"Metric {metric} not found for {ticker} in {year} Q{quarter}.")

In [13]:
def get_closest_filing_date(ticker: str, metric: str, units: str, date: str, quarterly: bool = False) -> str:
    """
    Get the closest filing date before a given date.

    Args:
        ticker (str): The ticker symbol for the company.
        metric (str): The metric to get.
        units (str): The units of the metric.
        date (str): The date to get the closest filing date for.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.

    Returns:
        str: The closest filing date.
    """

    with open(os.path.join(PROCESSED_DATA_DIR, f"{ticker}.json")) as f:
        data = orjson.loads(f.read())

    if quarterly:
        filing_dates = list(data[metric][units]["by_filing_date"]["quarterly"].keys())
    else:
        filing_dates = list(data[metric][units]["by_filing_date"]["annual"].keys())

    closest_date = None
    left = 0
    right = len(filing_dates) - 1
    while left <= right:
        mid = (left + right) // 2
        if filing_dates[mid] <= date:
            closest_date = filing_dates[mid]
            left = mid + 1
        else:
            right = mid - 1

    if closest_date is None:
        raise KeyError(f"Metric {metric} not found for {ticker} before {date}.")

    return closest_date

In [14]:
def get_metric(ticker: str, metric: str, units: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52):
    """
    Get a metric for a company from the most recent filing date before a given date.

    Args:
        ticker (str): The ticker symbol for the company.
        metric (str): The metric to get.
        units (str): The units of the metric.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The metric value.
    """

    with open(os.path.join(PROCESSED_DATA_DIR, f"{ticker}.json")) as f:
        data = orjson.loads(f.read())

    if query_date is None:
        query_date = datetime.now().strftime("%Y-%m-%d")

    filing_date = get_closest_filing_date(ticker, metric, units, query_date, quarterly)
    if datetime.strptime(query_date, "%Y-%m-%d") - datetime.strptime(filing_date, "%Y-%m-%d") > timedelta(weeks=tolerance):
        raise KeyError(f"Metric {metric} not found for {ticker} within {tolerance} weeks of {query_date}.")

    if quarterly:
        return data[metric][units]["by_filing_date"]["quarterly"][filing_date]["val"]
    else:
        return data[metric][units]["by_filing_date"]["annual"][filing_date]["val"]


In [15]:
def get_concept(
        ticker: str,
        concept: str,
        tags: List[str],
        units: str,
        query_date: str = None,
        quarterly: bool = False,
        tolerance: int = 5
    ) -> float:
    """
    Get a concept for a company by trying several different XRBL tags. Returns the first valid value found.

    Args:
        ticker (str): The ticker symbol for the company.
        concept (str): The name of the concept, strictly used for error messages.
        tags (List[str]): The XRBL tags to check for the concept.
        units (str): The units of the concept. Assumes the same units for all tags.
        query_date (str, optional): The date to get the concept for in Y-m-d format. Defaults to None, which gets the latest concept.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The concept value.
    """
    for tag in tags:
        try:
            return get_metric(ticker, tag, units, query_date, quarterly, tolerance)
        except KeyError:
            continue
    period = "quarterly" if quarterly else "annual"
    raise ValueError(f"Could not find {period} {concept} for {ticker} within {tolerance} weeks of {query_date}.")

### Methods to get specific financials from processed JSON data 

In [16]:
def get_shares_outstanding(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the shares outstanding for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The shares outstanding.
    """
    return get_concept(
        ticker,
        "shares outstanding",
        ["EntityCommonStockSharesOutstanding"],
        "shares",
        query_date,
        quarterly,
        tolerance
    )

In [17]:
def get_net_income(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the net income for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The net income.
    """
    tags = ["NetIncomeLoss", "ProfitLoss", "NetIncomeLossAvailableToCommonStockholdersBasic", "IncomeLossFromContinuingOperations"]
    return get_concept(
        ticker,
        "net income",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )
    

In [18]:
def get_interest_expense(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the interest expense for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The interest expense.
    """
    tags = ["InterestExpense", "InterestExpenseDebt", "InterestAndDebtExpense", "InterestExpenseBorrowings", "InterestIncomeExpenseNonoperatingNet"]
    return get_concept(
        ticker,
        "interest expense",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )

In [19]:
def get_tax_expense(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the tax expense for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The tax expense.
    """
    tags = ["IncomeTaxExpenseBenefit", "IncomeTaxExpenseBenefitContinuingOperations", "CurrentIncomeTaxExpenseBenefit"]
    return get_concept(
        ticker,
        "tax expense",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )

In [20]:
def get_revenue(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the revenue for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The revenue.
    """
    tags = ["Revenues", "SalesRevenueNet", "SalesRevenueGoodsNet", "SalesRevenueServicesNet", "SalesRevenueNetOfInterestExpense"]
    return get_concept(
        ticker,
        "revenue",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )

In [21]:
def get_cost_of_revenue(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the cost of revenue for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The cost of revenue.
    """
    tags = ["CostOfRevenue", "CostOfGoodsAndServicesSold", "CostOfGoodsSold", "CostOfServices"]
    return get_concept(
        ticker,
        "cost of revenue",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )

In [22]:
def get_operating_expenses(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the operating expenses for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The operating expenses.
    """
    tags = ["OperatingExpenses", "OperatingCostsAndExpenses"]
    return get_concept(
        ticker,
        "operating expenses",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )

In [23]:
def get_depreciation(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the depreciation for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The depreciation.
    """
    tags = ["Depreciation"]
    return get_concept(
        ticker,
        "depreciation",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )

In [24]:
def get_amortization(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the amortization for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The amortization.
    """
    tags = ["Amortization"]
    return get_concept(
        ticker,
        "amortization",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )

In [25]:
def get_depreciation_and_amortization(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the depreciation and amortization for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The depreciation and amortization.
    """
    tags = ["DepreciationAndAmortization", "DepreciationDepletionAndAmortization", "DepreciationAmortizationAndAccretionNet"]
    try:
        return get_concept(
            ticker,
            "depreciation and amortization",
            tags,
            "USD",
            query_date,
            quarterly,
            tolerance
        )
    except:
        try:
            depreciation = get_depreciation(ticker, query_date, quarterly, tolerance)
        except:
            depreciation = 0
        try:
            amortization = get_amortization(ticker, query_date, quarterly, tolerance)
        except:
            amortization = 0
        if depreciation == 0 and amortization == 0:
            raise ValueError("No depreciation or amortization found.")
        return depreciation + amortization


In [26]:
def get_current_debt(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the current debt for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The current debt.
    """
    tags = ["LongTermDebtCurrent"]
    return get_concept(
        ticker,
        "current debt",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )

In [27]:
def get_noncurrent_debt(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the noncurrent debt for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The noncurrent debt.
    """
    tags = ["LongTermDebtNoncurrent"]
    return get_concept(
        ticker,
        "noncurrent debt",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )

In [28]:
def get_total_debt(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the total debt for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The total debt.
    """
    current_debt = get_current_debt(ticker, query_date, quarterly, tolerance)
    noncurrent_debt = get_noncurrent_debt(ticker, query_date, quarterly, tolerance)
    return current_debt + noncurrent_debt

In [29]:
def get_capex(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the capital expenditures for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The capital expenditures.
    """
    tags = ["CapitalExpenditures", "PaymentsToAcquirePropertyPlantAndEquipment", "PaymentsToAcquireProductiveAssets"]
    return get_concept(
        ticker,
        "capital expenditures",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )

In [30]:
def get_current_assets(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the current assets for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The current assets.
    """
    tags = ["AssetsCurrent"]
    return get_concept(
        ticker,
        "current assets",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )

In [31]:
def get_current_liabilities(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the current liabilities for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The current liabilities.
    """
    tags = ["LiabilitiesCurrent"]
    return get_concept(
        ticker,
        "current liabilities",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )

In [32]:
def get_working_capital(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the working capital for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The working capital.
    """
    current_assets = get_current_assets(ticker, query_date, quarterly, tolerance)
    current_liabilities = get_current_liabilities(ticker, query_date, quarterly, tolerance)
    return current_assets - current_liabilities

In [33]:
def get_change_in_working_capital(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the change in working capital for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The change in working capital.
    """
    if query_date is None:
        query_date = datetime.today().strftime("%Y-%m-%d")
    one_year_ago = datetime.strptime(query_date, "%Y-%m-%d") - relativedelta.relativedelta(years=1)
    current_wc = get_working_capital(ticker, query_date, quarterly, tolerance)
    previous_wc = get_working_capital(ticker, one_year_ago.strftime("%Y-%m-%d"), quarterly, tolerance)
    return current_wc - previous_wc

In [34]:
def get_cash(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the cash for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The cash.
    """
    tags = ["CashAndCashEquivalentsAtCarryingValue"]
    return get_concept(
        ticker,
        "cash",
        tags,
        "USD",
        query_date,
        quarterly,
        tolerance
    )

In [35]:
def get_ebit(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the EBIT for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The EBIT.
    """
    try:
        net_income = get_net_income(ticker, query_date, quarterly, tolerance)
        interest_expense = get_interest_expense(ticker, query_date, quarterly, tolerance)
        tax_expense = get_tax_expense(ticker, query_date, quarterly, tolerance)
        return net_income + interest_expense + tax_expense
    except:
        revenue = get_revenue(ticker, query_date, quarterly, tolerance)
        cost_of_revenue = get_cost_of_revenue(ticker, query_date, quarterly, tolerance)
        operating_expenses = get_operating_expenses(ticker, query_date, quarterly, tolerance)
        return revenue - cost_of_revenue - operating_expenses

In [36]:
def get_ufcf(ticker: str, query_date: str = None, quarterly: bool = False, tolerance: int = 52) -> float:
    """
    Get the unlevered free cash flow for a company.

    Args:
        ticker (str): The ticker symbol for the company.
        query_date (str, optional): The date to get the metric for in Y-m-d format. Defaults to None, which gets the latest metric.
        quarterly (bool, optional): Whether to get the quarterly metric. Defaults to False.
        tolerance (int, optional): The max number of weeks to look back for a filing date. Defaults to 52.

    Returns:
        float: The unlevered free cash flow.
    """
    ebit = get_ebit(ticker, query_date, quarterly, tolerance)
    tax_expense = get_tax_expense(ticker, query_date, quarterly, tolerance)
    d_and_a = get_depreciation_and_amortization(ticker, query_date, quarterly, tolerance)
    capex = get_capex(ticker, query_date, quarterly, tolerance)
    change_in_wc = get_change_in_working_capital(ticker, query_date, quarterly, tolerance)
    return ebit - tax_expense + d_and_a - capex - change_in_wc